# Scanners

In [2]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=19)

<IB connected to 127.0.0.1:7497 clientId=19>

In [3]:
import pandas as pd
import datetime as dt
from scipy import interpolate
import numpy as np
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 40)

In [4]:
def winddown(time):
        if time < dt.time(9,30):
            return 0.7
        elif time < dt.time(9,45):
            return 0.6
        elif time < dt.time(10,30):
            return 0.5
        elif time < dt.time(11,30):
            return 0.4
        elif time < dt.time(13):
            return 0.3
        elif time <  dt.time(14,30):
            return 0.25
        elif time < dt.time(15,30):
            return 0.15
        elif time < dt.time(16):
            return 0.05
        else:
            return 0
   
   
def tte(expiry, data_collection_time):
    trading_days = np.busday_count( data_collection_time.date(), expiry )
    winddown_days = winddown(data_collection_time.time())
    return (trading_days+winddown_days)/253.0

def bs_implied_vol(F,K,t,kind,price):
    r = 0
    try:
        iv = implied_volatility(price, F,K,t,r, kind.lower())
    except:
        iv = 0
    return iv

def bs_delta(F,K,t,kind,iv):
    r = 0
    try:
        bs_delta = delta(kind.lower(), F,K,t,r, iv)
    except:
        bs_delta = 0
    return bs_delta




def convert_to_neptune(single_df):
    neptune_list_for_df = []
    for symbol in single_df.symbol.unique():
        for expiry in single_df.expiry.unique():
            single_df_temp = single_df[(single_df.expiry == expiry)& (single_df.symbol == symbol)]
            list_of_deltas = single_df_temp.mid_point_call_delta_otm.tolist()
            list_of_vols = single_df_temp.implied_vol_mid_point_otm.tolist()
            list_of_strikes = single_df_temp.strike.tolist()
        #     x = GetAnchorpointVol(list_of_deltas, list_of_vols, list_of_strikes, 'cubic')
            (vols, fake_strikes, anchors) = GetAnchorpointVol(list_of_deltas, list_of_vols, list_of_strikes)
            tte = single_df_temp.tte.iloc[0]
            und_price = single_df_temp.und_price.iloc[0]
            temp_dict = {'symbol' :  symbol, 'expiry' : expiry, 'tte' : tte, 'und_price': und_price, str(anchors[0]) : vols[0][()], str(anchors[1]) : vols[1][()], str(anchors[2]) : vols[2][()],
                         str(anchors[3]) : vols[3][()], str(anchors[4]) : vols[4][()], str(anchors[5]) : vols[5][()], 
                         str(anchors[6]) : vols[6][()], 
                         str(anchors[0])+'_str' : fake_strikes[0][()], str(anchors[1])+'_str' : fake_strikes[1][()], str(anchors[2])+'_str' : fake_strikes[2][()],
                         str(anchors[3])+'_str' : fake_strikes[3][()], str(anchors[4])+'_str' : fake_strikes[4][()], str(anchors[5])+'_str' : fake_strikes[5][()], 
                         str(anchors[6])+'_str' : fake_strikes[6][()]}
            neptune_list_for_df.append(temp_dict)
    neptune_df = pd.DataFrame(neptune_list_for_df)
    return neptune_df

            
def GetAnchorpointVol(list_of_deltas, list_of_vols, list_of_strikes, interpolation_type = 'linear'):
    anchors = [0.95,0.85,0.7,0.5,0.3,0.15,0.05]
    vol_spine = interpolate.interp1d(list_of_deltas, list_of_vols,bounds_error=False, fill_value=-1, kind = interpolation_type)
    strike_spine = interpolate.interp1d(list_of_deltas, list_of_strikes, bounds_error=False, fill_value=-1)
    vols = [vol_spine(anchors[0]), vol_spine(anchors[1]),vol_spine(anchors[2]),vol_spine(anchors[3]),
            vol_spine(anchors[4]),vol_spine(anchors[5]),vol_spine(anchors[6])] 
    fake_strikes = [strike_spine(anchors[0]), strike_spine(anchors[1]),strike_spine(anchors[2]),strike_spine(anchors[3]),
            strike_spine(anchors[4]),strike_spine(anchors[5]),strike_spine(anchors[6])] 
    return (vols, fake_strikes, anchors) 


def SaveRawData(raw_df):
    import os
    
    path = 'C:\\Users\\navsa\\Documents\\Python Projects\\ib_insync\\notebooks\\Data\\Raw\\'
    time_saved = dt.datetime.now()
    time_saved_string = time_saved.strftime("%Y-%m-%d_%H-%M")    
    file_name = time_saved_string + '.csv'
    raw_df.to_csv(path+file_name)
    
def SaveNeptuneData(neptune_df):
    import os
    
    path = 'C:\\Users\\navsa\\Documents\\Python Projects\\ib_insync\\notebooks\\Data\\Neptune\\'
    time_saved = dt.datetime.now()
    time_saved_string = time_saved.strftime("%Y-%m-%d_%H-%M")    
    file_name = time_saved_string + '.csv'
    neptune_df.to_csv(path+file_name)

## Basic Scanner

To create a scanner, create a `ScannerSubscription` to submit to the `reqScannerData` method. For any scanner to work, at least these three fields must be filled: `instrument` (the what), `locationCode` (the where), and `scanCode` (the ranking).

For example, to find the top ranked US stock percentage gainers:

In [4]:
#Get list of interesting companies

number_of_contracts = 2

sub = ScannerSubscription(
    instrument='STK', 
    locationCode='STK.US.MAJOR', 
    scanCode='TOP_TRADE_COUNT')
scanData = ib.reqScannerData(sub)
list_of_stock_contracts = [i.contractDetails.contract for i in scanData]
list_of_stock_contracts = list_of_stock_contracts[:number_of_contracts]

Error 162, reqId 5: Historical Market Data Service error message:API scanner subscription cancelled: 5


In [7]:
list_of_stock_contracts

[Contract(secType='STK', conId=484743832, symbol='ALF', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='ALF', tradingClass='SCM'),
 Contract(secType='STK', conId=460492611, symbol='WISH', exchange='SMART', currency='USD', localSymbol='WISH', tradingClass='NMS')]

In [5]:
contr = Stock('AMC', 'SMART', 'USD')
# contr = Contract(secType='FUT', conId=428520022)
# chains = ib.reqSecDefOptParams(contr.symbol, '', contr.secType, contr.conId)
list_of_stock_contracts = [contr]

In [ ]:
chains = ib.reqSecDefOptParams(contr.symbol, '', contr.secType, contr.conId)
util.df(chains)
chain = next(c for c in chains if c.tradingClass == contr.symbol and c.exchange == 'SMART')
print(chain)

In [5]:
# For each company:
# Request ticker for all options

#to do : have better way of selecting strikes to get full coverage but also fewer tickers

ticker_dict = {}
contract_dict = {}
for symbol_contract in list_of_stock_contracts:
    contract_dict[symbol_contract.symbol] = symbol_contract
    ib.qualifyContracts(symbol_contract)
    chains = ib.reqSecDefOptParams(symbol_contract.symbol, '', symbol_contract.secType, symbol_contract.conId)
    util.df(chains)
    chain = next(c for c in chains if c.tradingClass == symbol_contract.symbol and c.exchange == 'SMART')
    print(chain)
    strikes = [strike for strike in chain.strikes]
#         if strike > 200 and strike < 400]
#         and 0 < strike < amcValue + 20]
    expirations = sorted(exp for exp in chain.expirations)
    rights = ['P', 'C']

    contracts = [Option(symbol_contract.symbol, expiration, strike, right, 'SMART', tradingClass=symbol_contract.symbol)
        for right in rights
        for expiration in expirations
        for strike in strikes]

    contracts = ib.qualifyContracts(*contracts)
    print(len(contracts))
    tickers = ib.reqTickers(*contracts)
    print(tickers[0])
    ticker_dict[symbol_contract.symbol] = tickers
    

Error 200, reqId 33: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=13.5, right='P', exchange='SMART', tradingClass='AMC')


OptionChain(exchange='SMART', underlyingConId='140070600', tradingClass='AMC', multiplier='100', expirations=['20210625', '20210702', '20210709', '20210716', '20210723', '20210730', '20210806', '20210820', '20210917', '20211217', '20220121', '20220715', '20230120'], strikes=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5, 55.0, 55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0, 60.5, 61.0, 61.5, 62.0, 62.5, 63.0, 63.5, 64.0, 64.5, 65.0, 65.5, 66.0, 66.5, 67.0, 67.5, 68.0, 68.5, 69.0, 69.5, 70.0, 71.0, 72.0,

Error 200, reqId 35: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=14.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 39: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=16.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 41: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=17.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 37: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=15.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 43: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=18.5,

Error 200, reqId 201: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210702', strike=18.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 199: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210702', strike=17.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 205: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210702', strike=20.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 203: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210702', strike=19.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 207: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210702', strike=

Error 200, reqId 330: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210709', strike=4.0, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 329: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210709', strike=3.5, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 328: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210709', strike=3.0, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 332: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210709', strike=5.0, right='P', exchange='SMART', tradingClass='AMC')
Error 200, reqId 331: No security definition has been found for the request, contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20210709', strike=4.5,

...

Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220121', strike=96.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=0.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=1.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=1.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=2.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=2.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=3.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown cont

Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=47.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=48.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=48.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=49.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=49.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=50.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=51.0, right='C', exchange='SMART', tradingClass='AMC')
Unknow

Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=92.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=93.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=94.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20220715', strike=96.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=6.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=6.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=7.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown c

Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=51.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=51.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=52.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=52.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=53.0, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=53.5, right='C', exchange='SMART', tradingClass='AMC')
Unknown contract: Option(symbol='AMC', lastTradeDateOrContractMonth='20230120', strike=54.0, right='C', exchange='SMART', tradingClass='AMC')
Unknow

2126
Ticker(contract=Option(conId=489989977, symbol='AMC', lastTradeDateOrContractMonth='20210625', strike=0.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMC   210625P00000500', tradingClass='AMC'), time=datetime.datetime(2021, 6, 25, 15, 53, 59, 787968, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=0.01, askSize=2732, close=0.0, bidGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=54.29999923706055), askGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=0.009999999776482582, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=54.29999923706055), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=54.29999923706055), modelGreeks=OptionComputation(impliedVol=23.01400187039636, delta=-1.4190206096873977e-27, optPrice=4.155573031792236e-27, pvDividend=0.0, gamma=5.0956

In [6]:
# df with columns : symbol	expiry	strike	kind	und_price	bid_price	ask_price	mid_price	last_price	volume	ib_bid_delta


row_list_for_df = []
for ticker_symbol,ticker_list in ticker_dict.items():
    for ticker in ticker_list:
        if ticker.modelGreeks == None:
            continue
        print(ticker_symbol, ticker.contract.lastTradeDateOrContractMonth,ticker.contract.strike )
        symbol = ticker_symbol
        expiry = dt.datetime.strptime(ticker.contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        strike = ticker.contract.strike
        kind = ticker.contract.right
        und_price = ticker.modelGreeks.undPrice
        bid_price = ticker.bid
        ask_price = ticker.ask
        mid_price = (bid_price+ask_price)/2
        last_price = ticker.last
        volume = ticker.volume
        ib_bid_delta = ticker.bidGreeks.delta    
        data_collection_time = dt.datetime.now()
        ticker_dict_temp = {'data_collection_time' : data_collection_time, 'symbol' : symbol , 'expiry' : expiry, 'strike' : strike, 'kind': kind, 'und_price' : und_price,
                       'bid_price': bid_price, 'ask_price' : ask_price, 'mid_price' : mid_price,
                       'last_price' : last_price, 'volume' : volume, 'ib_bid_delta' : ib_bid_delta }
        row_list_for_df.append(ticker_dict_temp)
df = pd.DataFrame(row_list_for_df)
SaveRawData(df)

AMC 20210625 0.5
AMC 20210625 1.0
AMC 20210625 1.5
AMC 20210625 2.0
AMC 20210625 2.5
AMC 20210625 3.0
AMC 20210625 3.5
AMC 20210625 4.0
AMC 20210625 4.5
AMC 20210625 5.0
AMC 20210625 5.5
AMC 20210625 6.0
AMC 20210625 6.5
AMC 20210625 7.0
AMC 20210625 7.5
AMC 20210625 8.0
AMC 20210625 8.5
AMC 20210625 9.0
AMC 20210625 9.5
AMC 20210625 10.0
AMC 20210625 10.5
AMC 20210625 11.0
AMC 20210625 11.5
AMC 20210625 12.0
AMC 20210625 12.5
AMC 20210625 13.0
AMC 20210625 14.0
AMC 20210625 15.0
AMC 20210625 16.0
AMC 20210625 17.0
AMC 20210625 18.0
AMC 20210625 19.0
AMC 20210625 20.0
AMC 20210625 21.0
AMC 20210625 22.0
AMC 20210625 23.0
AMC 20210625 24.0
AMC 20210625 25.0
AMC 20210625 26.0
AMC 20210625 27.0
AMC 20210625 28.0
AMC 20210625 29.0
AMC 20210625 30.0
AMC 20210625 31.0
AMC 20210625 32.0
AMC 20210625 33.0
AMC 20210625 34.0
AMC 20210625 35.0
AMC 20210625 36.0
AMC 20210625 37.0
AMC 20210625 38.0
AMC 20210625 39.0
AMC 20210625 40.0
AMC 20210625 41.0
AMC 20210625 42.0
AMC 20210625 43.0
AMC 2021062

AMC 20210730 38.0
AMC 20210730 39.0
AMC 20210730 40.0
AMC 20210730 41.0
AMC 20210730 42.0
AMC 20210730 43.0
AMC 20210730 43.5
AMC 20210730 44.0
AMC 20210730 44.5
AMC 20210730 45.0
AMC 20210730 45.5
AMC 20210730 46.0
AMC 20210730 46.5
AMC 20210730 47.0
AMC 20210730 47.5
AMC 20210730 48.0
AMC 20210730 48.5
AMC 20210730 49.0
AMC 20210730 49.5
AMC 20210730 50.0
AMC 20210730 50.5
AMC 20210730 51.0
AMC 20210730 51.5
AMC 20210730 52.0
AMC 20210730 52.5
AMC 20210730 53.0
AMC 20210730 53.5
AMC 20210730 54.0
AMC 20210730 54.5
AMC 20210730 55.0
AMC 20210730 56.0
AMC 20210730 57.0
AMC 20210730 58.0
AMC 20210730 59.0
AMC 20210730 60.0
AMC 20210730 61.0
AMC 20210730 62.0
AMC 20210730 63.0
AMC 20210730 64.0
AMC 20210730 65.0
AMC 20210730 66.0
AMC 20210730 67.0
AMC 20210730 68.0
AMC 20210730 69.0
AMC 20210730 70.0
AMC 20210730 71.0
AMC 20210730 72.0
AMC 20210730 73.0
AMC 20210730 74.0
AMC 20210730 75.0
AMC 20210730 76.0
AMC 20210730 77.0
AMC 20210730 78.0
AMC 20210730 79.0
AMC 20210730 80.0
AMC 202107

AMC 20210625 21.0
AMC 20210625 22.0
AMC 20210625 23.0
AMC 20210625 24.0
AMC 20210625 25.0
AMC 20210625 26.0
AMC 20210625 27.0
AMC 20210625 28.0
AMC 20210625 29.0
AMC 20210625 30.0
AMC 20210625 31.0
AMC 20210625 32.0
AMC 20210625 33.0
AMC 20210625 34.0
AMC 20210625 35.0
AMC 20210625 36.0
AMC 20210625 37.0
AMC 20210625 38.0
AMC 20210625 39.0
AMC 20210625 40.0
AMC 20210625 41.0
AMC 20210625 42.0
AMC 20210625 43.0
AMC 20210625 44.0
AMC 20210625 45.0
AMC 20210625 46.0
AMC 20210625 47.0
AMC 20210625 48.0
AMC 20210625 49.0
AMC 20210625 50.0
AMC 20210625 51.0
AMC 20210625 52.0
AMC 20210625 53.0
AMC 20210625 54.0
AMC 20210625 55.0
AMC 20210625 56.0
AMC 20210625 57.0
AMC 20210625 58.0
AMC 20210625 59.0
AMC 20210625 60.0
AMC 20210625 61.0
AMC 20210625 62.0
AMC 20210625 63.0
AMC 20210625 64.0
AMC 20210625 65.0
AMC 20210625 66.0
AMC 20210625 67.0
AMC 20210625 68.0
AMC 20210625 69.0
AMC 20210625 70.0
AMC 20210625 71.0
AMC 20210625 72.0
AMC 20210625 73.0
AMC 20210625 74.0
AMC 20210625 75.0
AMC 202106

AMC 20210820 38.0
AMC 20210820 39.0
AMC 20210820 40.0
AMC 20210820 41.0
AMC 20210820 42.0
AMC 20210820 43.0
AMC 20210820 44.0
AMC 20210820 45.0
AMC 20210820 46.0
AMC 20210820 47.0
AMC 20210820 48.0
AMC 20210820 49.0
AMC 20210820 50.0
AMC 20210820 55.0
AMC 20210820 60.0
AMC 20210820 65.0
AMC 20210820 70.0
AMC 20210820 75.0
AMC 20210820 80.0
AMC 20210820 85.0
AMC 20210820 90.0
AMC 20210820 95.0
AMC 20210820 100.0
AMC 20210820 105.0
AMC 20210820 110.0
AMC 20210820 115.0
AMC 20210820 120.0
AMC 20210820 125.0
AMC 20210820 130.0
AMC 20210820 135.0
AMC 20210820 140.0
AMC 20210820 145.0
AMC 20210917 0.5
AMC 20210917 1.0
AMC 20210917 1.5
AMC 20210917 2.0
AMC 20210917 2.5
AMC 20210917 3.0
AMC 20210917 3.5
AMC 20210917 4.0
AMC 20210917 4.5
AMC 20210917 5.0
AMC 20210917 5.5
AMC 20210917 6.0
AMC 20210917 7.0
AMC 20210917 8.0
AMC 20210917 9.0
AMC 20210917 10.0
AMC 20210917 11.0
AMC 20210917 12.0
AMC 20210917 13.0
AMC 20210917 14.0
AMC 20210917 15.0
AMC 20210917 16.0
AMC 20210917 17.0
AMC 20210917 18

In [7]:
#add tte, iv and delta
df['tte'] = df.apply(lambda x : tte(x.expiry, x.data_collection_time), axis = 1)
df['implied_vol_mid_point'] = df.apply(lambda x : bs_implied_vol(x.und_price, x.strike, x.tte, x.kind, x.mid_price), axis= 1)
df['mid_point_delta'] =  df.apply(lambda x : bs_delta(x.und_price, x.strike, x.tte, x.kind, x.implied_vol_mid_point), axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes\__init__.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / denominator


In [8]:
#restructure df to get call and put in same row
df_calls = df[df.kind == 'C']
df_puts = df[df.kind == 'P']
single_df = pd.merge(df_calls, df_puts, on = ['symbol', 'expiry', 'strike'], suffixes = ("_call", "_put"))
single_df['und_price'] = single_df.und_price_call
single_df['tte'] = single_df.tte_call
single_df = single_df.drop(columns = ['kind_call', 'kind_put', 'und_price_put', 'tte_put', 'und_price_call', 'tte_call' ])


In [9]:

#restructure to get otm option
single_df['bid_price_otm'] = single_df.apply(lambda x : x.bid_price_call if x.mid_point_delta_call < 0.5 else x.bid_price_put, axis = 1)
single_df['ask_price_otm'] = single_df.apply(lambda x : x.ask_price_call if x.mid_point_delta_call < 0.5 else x.ask_price_put, axis = 1)
single_df['mid_price_otm'] = single_df.apply(lambda x : x.mid_price_call if x.mid_point_delta_call < 0.5 else x.mid_price_put, axis = 1)
single_df['implied_vol_mid_point_otm'] = single_df.apply(lambda x : x.implied_vol_mid_point_call if x.mid_point_delta_call < 0.5 else x.implied_vol_mid_point_put, axis = 1)
single_df['mid_point_delta_otm'] = single_df.apply(lambda x : x.mid_point_delta_call if x.mid_point_delta_call < 0.5 else x.mid_point_delta_put, axis = 1)
single_df['mid_point_call_delta_otm'] = single_df.apply(lambda x : x.mid_point_delta_otm if x.mid_point_delta_otm > 0 else (1 - x.mid_point_delta_otm*-1), axis = 1)


In [10]:
#filter out the columns
columns_to_filter = ['symbol', 'expiry', 'strike','und_price', 'tte', 'bid_price_otm',
       'ask_price_otm', 'mid_price_otm', 'implied_vol_mid_point_otm',
       'mid_point_delta_otm', 'mid_point_call_delta_otm']
single_df = single_df[columns_to_filter]
single_df = single_df.dropna()

In [11]:
single_df

,symbol,expiry,strike,und_price,tte,bid_price_otm,ask_price_otm,mid_price_otm,implied_vol_mid_point_otm,mid_point_delta_otm,mid_point_call_delta_otm
0,AMC,2021-06-25,0.5,54.289799,0.001186,-1.00,0.01,-0.495,0.000000,0.000000,1.000000
1,AMC,2021-06-25,1.0,54.289799,0.001186,-1.00,0.01,-0.495,0.000000,0.000000,1.000000
2,AMC,2021-06-25,1.5,54.289799,0.001186,-1.00,0.01,-0.495,0.000000,0.000000,1.000000
3,AMC,2021-06-25,2.0,54.289799,0.001186,-1.00,0.01,-0.495,0.000000,0.000000,1.000000
4,AMC,2021-06-25,2.5,54.289799,0.001186,-1.00,0.01,-0.495,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
1048,AMC,2023-01-20,125.0,54.348999,1.621739,91.40,93.40,92.400,1.265312,-0.386633,0.613367
1049,AMC,2023-01-20,130.0,54.348999,1.621739,95.30,97.90,96.600,1.252358,-0.401328,0.598672
1050,AMC,2023-01-20,135.0,54.348999,1.621739,99.95,102.90,101.425,1.262119,-0.406375,0.593625
1051,AMC,2023-01-20,140.0,54.348999,1.621739,104.40,107.90,106.150,1.267414,-0.412902,0.587098


In [12]:
#get neptune format
neptune_df = convert_to_neptune(single_df)
SaveNeptuneData(neptune_df)

In [13]:
neptune_df

,symbol,expiry,tte,und_price,0.95,0.85,0.7,0.5,0.3,0.15,0.05,0.95_str,0.85_str,0.7_str,0.5_str,0.3_str,0.15_str,0.05_str
0,AMC,2021-06-25,0.001186,54.289799,2.558738,1.706477,1.497398,1.456288,1.512380,1.682789,2.255848,47.156530,51.167555,52.923401,54.393947,55.928315,57.823853,61.936720
1,AMC,2021-07-02,0.020949,54.331001,1.844741,1.611588,1.611397,1.673910,1.817578,2.143556,3.060427,36.278524,43.827750,49.382020,55.947445,64.545735,78.606814,124.218526
2,AMC,2021-07-09,0.040711,54.310001,1.957360,1.712985,1.725931,1.779447,1.917976,2.177444,-1.000000,30.657173,40.289189,48.073179,57.924701,71.706654,94.299375,-1.000000
3,AMC,2021-07-16,0.060474,54.303001,2.091411,1.860961,1.876362,1.959346,2.112710,2.448392,-1.000000,26.586914,37.510667,47.410007,60.970937,81.665717,121.507158,-1.000000
4,AMC,2021-07-23,0.080237,54.279999,2.029328,1.863611,1.923268,2.043560,2.173783,-1.000000,-1.000000,24.877876,36.108794,47.327947,64.188915,90.601751,-1.000000,-1.000000
5,AMC,2021-07-30,0.100000,54.270000,1.978090,1.890825,1.999558,2.110792,2.217414,-1.000000,-1.000000,23.588815,34.928852,47.589810,67.808317,100.189149,-1.000000,-1.000000
6,AMC,2021-08-06,0.119763,54.240002,2.038833,1.940805,2.045403,2.150471,2.246775,-1.000000,-1.000000,21.668297,33.863060,48.140472,71.595387,110.315137,-1.000000,-1.000000
7,AMC,2021-08-20,0.159289,54.229000,1.915107,1.880824,1.995677,2.083466,2.147963,-1.000000,-1.000000,20.664543,33.027417,49.064351,76.676612,122.803343,-1.000000,-1.000000
8,AMC,2021-09-17,0.238340,54.230000,1.837111,1.815026,1.916491,1.977655,-1.000000,-1.000000,-1.000000,18.547829,32.067255,51.468800,86.442838,-1.000000,-1.000000,-1.000000
9,AMC,2021-12-17,0.495257,54.232800,1.631155,1.586011,1.651540,1.626015,-1.000000,-1.000000,-1.000000,15.859513,31.800860,57.968854,104.393897,-1.000000,-1.000000,-1.000000


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.


In [107]:
# single_df = single_df.dropna()

In [30]:
single_df['bid_price_otm'] = single_df.apply(lambda x : x.bid_price_call if x.mid_point_delta_call < 0.5 else x.bid_price_put, axis = 1)
single_df['ask_price_otm'] = single_df.apply(lambda x : x.ask_price_call if x.mid_point_delta_call < 0.5 else x.ask_price_put, axis = 1)
single_df['mid_price_otm'] = single_df.apply(lambda x : x.mid_price_call if x.mid_point_delta_call < 0.5 else x.mid_price_put, axis = 1)
single_df['implied_vol_mid_point_otm'] = single_df.apply(lambda x : x.implied_vol_mid_point_call if x.mid_point_delta_call < 0.5 else x.implied_vol_mid_point_put, axis = 1)
single_df['mid_point_delta_otm'] = single_df.apply(lambda x : x.mid_point_delta_call if x.mid_point_delta_call < 0.5 else x.mid_point_delta_put, axis = 1)


In [33]:
columns_to_filter = ['symbol', 'expiry', 'strike','und_price', 'tte', 'bid_price_otm',
       'ask_price_otm', 'mid_price_otm', 'implied_vol_mid_point_otm',
       'mid_point_delta_otm']
single_df = single_df[columns_to_filter]

In [50]:
single_df

,symbol,expiry,strike,und_price,tte,bid_price_otm,ask_price_otm,mid_price_otm,implied_vol_mid_point_otm,mid_point_delta_otm,mid_point_call_delta_otm
0,GME,2021-06-11,202.5,272.000000,0.005929,0.67,0.90,0.785,2.284654,-0.038284,0.961716
1,GME,2021-06-11,205.0,272.000000,0.005929,0.73,0.87,0.800,2.213950,-0.040059,0.959941
2,GME,2021-06-11,207.5,272.000000,0.005929,0.76,1.01,0.885,2.179825,-0.044328,0.955672
3,GME,2021-06-11,210.0,272.000000,0.005929,0.82,1.08,0.950,2.132241,-0.048068,0.951932
4,GME,2021-06-11,212.5,272.000000,0.005929,0.91,1.18,1.045,2.095774,-0.052976,0.947024
5,GME,2021-06-11,215.0,272.000000,0.005929,1.02,1.20,1.110,2.043148,-0.057070,0.942930
6,GME,2021-06-11,217.5,272.000000,0.005929,1.02,1.33,1.175,1.988710,-0.061373,0.938627
7,GME,2021-06-11,220.0,272.000000,0.005929,1.19,1.30,1.245,1.934438,-0.066074,0.933926
8,GME,2021-06-11,222.5,272.000000,0.005929,1.24,1.50,1.370,1.897498,-0.072864,0.927136
9,GME,2021-06-11,225.0,272.000000,0.005929,1.36,1.75,1.555,1.875621,-0.081856,0.918144


GME 2021-06-11 {'symbol': 'GME', 'expiry': '2021-06-11', 'tte': 0.005928853754940711, 'und_price': 272.0, '0.95': 2.1178871868974105, '0.85': 1.7184274046073817, '0.7': 1.6315400108958726, '0.5': 1.6285432046933155, '0.3': 1.651157378796273, '0.15': 1.8337358189378337, '0.05': 2.4189406214157363}
GME 2021-06-18 {'symbol': 'GME', 'expiry': '2021-06-18', 'tte': 0.025691699604743084, 'und_price': 272.010009765625, '0.95': -1.0, '0.85': 1.5093168497225078, '0.7': 1.4714000991044949, '0.5': 1.5115603254597734, '0.3': 1.6201124340390323, '0.15': 1.8114048585076112, '0.05': -1.0}
GME 2021-06-25 {'symbol': 'GME', 'expiry': '2021-06-25', 'tte': 0.045454545454545456, 'und_price': 272.010009765625, '0.95': -1.0, '0.85': -1.0, '0.7': 1.4843340055183594, '0.5': 1.5351092449147463, '0.3': 1.6337733729847976, '0.15': -1.0, '0.05': -1.0}
GME 2021-07-02 {'symbol': 'GME', 'expiry': '2021-07-02', 'tte': 0.06521739130434782, 'und_price': 272.010009765625, '0.95': -1.0, '0.85': -1.0, '0.7': 1.4856782490971

,symbol,expiry,tte,und_price,0.95,0.85,0.7,0.5,0.3,0.15,0.05
0,GME,2021-06-11,0.005929,272.000000,2.117887,1.718427,1.631540,1.628543,1.651157,1.833736,2.418941
1,GME,2021-06-18,0.025692,272.010010,-1.000000,1.509317,1.471400,1.511560,1.620112,1.811405,-1.000000
2,GME,2021-06-25,0.045455,272.010010,-1.000000,-1.000000,1.484334,1.535109,1.633773,-1.000000,-1.000000
3,GME,2021-07-02,0.065217,272.010010,-1.000000,-1.000000,1.485678,1.528854,1.643847,-1.000000,-1.000000
4,GME,2021-07-09,0.084980,272.075012,-1.000000,-1.000000,1.449308,1.492450,1.603324,-1.000000,-1.000000
5,GME,2021-07-16,0.104743,273.579987,-1.000000,-1.000000,1.457439,1.488895,-1.000000,-1.000000,-1.000000
6,GME,2021-07-23,0.124506,272.000000,-1.000000,-1.000000,1.447049,1.496347,-1.000000,-1.000000,-1.000000
7,GME,2021-07-30,0.144269,273.579987,-1.000000,-1.000000,1.509900,1.583458,-1.000000,-1.000000,-1.000000
8,GME,2021-08-20,0.203557,272.000000,-1.000000,-1.000000,1.431914,1.503388,-1.000000,-1.000000,-1.000000
9,GME,2021-09-17,0.282609,272.000000,-1.000000,-1.000000,1.425805,1.464948,-1.000000,-1.000000,-1.000000


In [37]:
single_df = pd.read_csv('gme_data.csv')

In [38]:
single_df

,Unnamed: 0,symbol,expiry,strike,bid_price_call,ask_price_call,mid_price_call,last_price_call,volume_call,ib_bid_delta_call,...,bid_price_put,ask_price_put,mid_price_put,last_price_put,volume_put,ib_bid_delta_put,implied_vol_mid_point_put,mid_point_delta_put,und_price,tte
0,0,GME,2021-06-11,202.5,69.10,71.30,70.200,NaN,5.0,NaN,...,0.67,0.90,0.785,0.70,19.0,-0.023454,2.284654,-0.038284,272.0,0.005929
1,1,GME,2021-06-11,205.0,66.65,68.85,67.750,69.2,3.0,NaN,...,0.73,0.87,0.800,0.81,44.0,-0.020999,2.213950,-0.040059,272.0,0.005929
2,2,GME,2021-06-11,207.5,64.25,66.40,65.325,NaN,NaN,NaN,...,0.76,1.01,0.885,1.01,13.0,-0.023661,2.179825,-0.044328,272.0,0.005929
3,3,GME,2021-06-11,210.0,61.80,64.05,62.925,70.0,2.0,0.996436,...,0.82,1.08,0.950,0.90,148.0,-0.026428,2.132241,-0.048068,272.0,0.005929
4,4,GME,2021-06-11,212.5,59.35,61.70,60.525,NaN,NaN,0.990754,...,0.91,1.18,1.045,1.00,5.0,-0.037219,2.095774,-0.052976,272.0,0.005929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,492,GME,2023-01-20,350.0,116.50,134.50,125.500,NaN,NaN,0.680225,...,203.50,222.50,213.000,NaN,NaN,-0.296566,1.180403,-0.275025,272.0,1.666008
493,493,GME,2023-01-20,360.0,115.00,133.00,124.000,NaN,NaN,0.673787,...,211.50,231.00,221.250,NaN,NaN,-0.301991,1.180390,-0.281232,272.0,1.666008
494,494,GME,2023-01-20,370.0,113.00,131.00,122.000,NaN,NaN,0.666732,...,219.50,239.00,229.250,NaN,NaN,-0.308815,1.177906,-0.288021,272.0,1.666008
495,495,GME,2023-01-20,380.0,111.00,129.50,120.250,NaN,NaN,0.659673,...,228.00,247.00,237.500,NaN,NaN,-0.312933,1.175780,-0.295120,272.0,1.666008


In [69]:
single_df_temp = single_df[single_df.expiry == '2021-06-11' ]

In [73]:
x

([array(2.10324614),
  array(1.76782563),
  array(1.62241261),
  array(1.62473318),
  array(1.65020376),
  array(1.83078003),
  array(2.41564043)],
 [array(211.2759504),
  array(238.58798558),
  array(256.75669125),
  array(274.14291477),
  array(293.12475954),
  array(318.2139543),
  array(376.10642416)],
 [0.95, 0.85, 0.7, 0.5, 0.3, 0.15, 0.05])

In [74]:
y

([array(2.09689455),
  array(1.75570672),
  array(1.62004668),
  array(1.62279915),
  array(1.65115738),
  array(1.83373582),
  array(2.41894062)],
 [array(211.2759504),
  array(238.58798558),
  array(256.75669125),
  array(274.14291477),
  array(293.12475954),
  array(318.2139543),
  array(376.10642416)],
 [0.95, 0.85, 0.7, 0.5, 0.3, 0.15, 0.05])

In [25]:
  greeks_dict['delta'] = ticker.modelGreeks.delta
#         print(type(position), type(ticker.modelGreeks.undPrice) , float(contract.multiplier))
        greeks_dict['cash_delta'] = position * ticker.modelGreeks.delta * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['expiry'] = dt.datetime.strptime(contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        greeks_dict['strike'] = contract.strike
        greeks_dict['kind'] = contract.right
        greeks_dict['vega'] = ticker.modelGreeks.vega
        greeks_dict['gamma'] = ticker.modelGreeks.gamma
        greeks_dict['theta'] = ticker.modelGreeks.theta
        greeks_dict['weighted_vega'] = GetWeightedVega(ticker.modelGreeks.vega, 1)
        greeks_dict['cash_gamma'] = position * ticker.modelGreeks.gamma * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['avg_cost'] = avg_cost
        greeks_dict['current_price'] = ticker.modelGreeks.optPrice
#     print(greeks_dict)
df = pd.DataFrame({'symbol' : [], 'expiry' : [], 'call_bid_price': [], 'put_bid_price' : [], 
                   'call_offer_price' : []})




chains = ib.reqSecDefOptParams(es.symbol, '', es.secType, es.conId)
# util.df(chains)
# chain = next(c for c in chains if c.tradingClass == es.symbol and c.exchange == 'SMART')
# print(chain)
# strikes = [strike for strike in chain.strikes
#     if strike % 100 == 0]
# #         and 0 < strike < amcValue + 20]
# expirations = sorted(exp for exp in chain.expirations)
# rights = ['P', 'C']

# contracts = [Option(symbol_contract.symbol, expiration, strike, right, 'SMART', tradingClass=symbol_contract.symbol)
#     for right in rights
#     for expiration in expirations
#     for strike in strikes]

# contracts = ib.qualifyContracts(*contracts)
# print(len(contracts))
# tickers = ib.reqTickers(*contracts)
# print(tickers[0])
# # ticker_dict[symbol_contract.symbol] = tickers
    a = Option('ES','20210618', 4000, 'C', 'GLOBEX', tradingClass = 'ES')
b = [a]

In [26]:
rows_list = []
for row in input_rows:

        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update(blah..) 

        rows_list.append(dict1)

df = pd.DataFrame(rows_list)    

,symbol,expiry,call_bid_price,put_bid_price,call_offer_price


In [18]:
contract[0]

TypeError: 'Option' object is not subscriptable

In [7]:
list_of_stock_contracts

[Contract(secType='STK', conId=464040647, symbol='CLOV', exchange='SMART', currency='USD', localSymbol='CLOV', tradingClass='NMS'),
 Contract(secType='STK', conId=140070600, symbol='AMC', exchange='SMART', currency='USD', localSymbol='AMC', tradingClass='AMC'),
 Contract(secType='STK', conId=460492611, symbol='WISH', exchange='SMART', currency='USD', localSymbol='WISH', tradingClass='NMS'),
 Contract(secType='STK', conId=131217639, symbol='BB', exchange='SMART', currency='USD', localSymbol='BB', tradingClass='BB'),
 Contract(secType='STK', conId=283766043, symbol='NEXT', exchange='SMART', currency='USD', localSymbol='NEXT', tradingClass='SCM'),
 Contract(secType='STK', conId=90655688, symbol='WEN', exchange='SMART', currency='USD', localSymbol='WEN', tradingClass='NMS'),
 Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', currency='USD', localSymbol='AAPL', tradingClass='NMS'),
 Contract(secType='STK', conId=215119556, symbol='WKHS', exchange='SMART', currency='USD'

*The displayed error 162 can be ignored*

The scanner returns a list of contract details, without current market data (this can be obtained via seperate market data requests).

## Filtering scanner results, the old way

The `ScannerSubscription` object has addional parameters that can be set to filter the results, such as `abovePrice`, `aboveVolume`, `marketCapBelow` or `spRatingAbove`. 

For example, to reuse the previous `sub` and query only for stocks with a price above 200 dollar:

In [3]:
sub.abovePrice = 200
scanData = ib.reqScannerData(sub)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3


['FVRR', 'BA', 'AMGN', 'VRTX', 'LMT']


## Filtering, the new way

In the new way there is a truly vast number of parameters available to use for filtering.
These new scanner parameters map directly to the options available through the TWS "Advanced Market Scanner." The parameters
are dynamically available from a huge XML document that is returned by ``reqScannerParameters``:

In [4]:
xml = ib.reqScannerParameters()

print(len(xml), 'bytes')

Error 162, reqId 4: Historical Market Data Service error message:API scanner subscription cancelled: 4


2000323 bytes


To view the XML in a web browser:

In [5]:
path = 'scanner_parameters.xml'
with open(path, 'w') as f:
    f.write(xml)

import webbrowser
webbrowser.open(path)

True

In [6]:
# parse XML document
import xml.etree.ElementTree as ET
tree = ET.fromstring(xml)

# find all tags that are available for filtering
tags = [elem.text for elem in tree.findall('.//AbstractField/code')]
print(len(tags), 'tags:')
print(tags)

1352 tags:
['priceAbove', 'priceBelow', 'usdPriceAbove', 'usdPriceBelow', 'volumeAbove', 'usdVolumeAbove', 'usdVolumeBelow', 'avgVolumeAbove', 'avgVolumeBelow', 'avgUsdVolumeAbove', 'avgUsdVolumeBelow', 'ihNumSharesInsiderAbove', 'ihNumSharesInsiderBelow', 'ihInsiderOfFloatPercAbove', 'ihInsiderOfFloatPercBelow', 'iiNumSharesInstitutionalAbove', 'iiNumSharesInstitutionalBelow', 'iiInstitutionalOfFloatPercAbove', 'iiInstitutionalOfFloatPercBelow', 'marketCapAbove1e6', 'marketCapBelow1e6', 'moodyRatingAbove', 'moodyRatingBelow', 'spRatingAbove', 'spRatingBelow', 'ratingsRelation', 'bondCreditRating', 'maturityDateAbove', 'maturityDateBelow', 'currencyLike', 'excludeConvertible', 'couponRateAbove', 'couponRateBelow', 'optVolumeAbove', 'optVolumeBelow', 'avgOptVolumeAbove', 'optVolumePCRatioAbove', 'optVolumePCRatioBelow', 'impVolatAbove', 'impVolatBelow', 'impVolatOverHistAbove', 'impVolatOverHistBelow', 'imbalanceAbove', 'imbalanceBelow', 'displayImbalanceAdvRatioAbove', 'displayImbalanc

Notice how ``abovePrice`` is now called ``priceAbove``...

Using three of these filter tags, let's perform a query to find all US stocks that went up 20% and have a current price between 5 and 50 dollar, sorted by percentage gain:

In [7]:
sub = ScannerSubscription(
    instrument='STK',
    locationCode='STK.US.MAJOR',
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", "20"),
    TagValue('priceAbove', 5),
    TagValue('priceBelow', 50)]

# the tagValues are given as 3rd argument; the 2nd argument must always be an empty list
# (IB has not documented the 2nd argument and it's not clear what it does)
scanData = ib.reqScannerData(sub, [], tagValues)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

[]


Any scanner query that TWS can do can alse be done through the API. The `scanCode` parameter maps directly to the "Parameter" window in the TWS "Advanced Market Scanner." We can verify this by printing out the `scanCode` values available:

In [8]:
scanCodes = [e.text for e in tree.findall('.//scanCode')]

print(len(scanCodes), 'scan codes, showing the ones starting with "TOP":')
print([sc for sc in scanCodes if sc.startswith('TOP')])

762 scan codes, showing the ones starting with "TOP":
['TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_COUNT', 'TOP_TRADE_RATE', 'TOP_PRICE_RANGE', 'TOP_VOLUME_RATE', 'TOP_OPEN_PERC_GAIN', 'TOP_OPEN_PERC_LOSE', 'TOP_AFTER_HOURS_PERC_GAIN', 'TOP_AFTER_HOURS_PERC_LOSE', 'TOP_OPT_IMP_VOLAT_GAIN', 'TOP_OPT_IMP_VOLAT_LOSE', 'TOP_STOCK_BUY_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_IMBALANCE_ADV_RATIO', 'TOP_STOCK_BUY_REG_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_REG_IMBALANCE_ADV_RATIO', 'TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_RATE', 'TOP_PERC_GAIN']


Queries are not limited to stocks. To get a list of all supported instruments:

In [9]:
instrumentTypes = set(e.text for e in tree.findall('.//Instrument/type'))
print(instrumentTypes)

{'BOND.AGNCY', 'ETF.FI.US', 'STOCK.EU', 'SSF.NA', 'BOND.MUNI', 'BOND.GOVT', 'BOND.CD', 'SSF.HK', 'WAR.EU', 'STOCK.HK', 'SSF.EU', 'IND.EU', 'IND.HK', 'FUND.ALL', 'STK', 'FUT.HK', 'BOND', 'FUT.US', 'STOCK.NA', 'BOND.GOVT.NON-US', 'PORTSTK', 'Global', 'IND.US', 'FUT.EU', 'STOCK.ME', 'SLB.US', 'NATCOMB', 'FUT.NA', 'ETF.EQ.US'}


To find all location codes:

In [10]:
locationCodes = [e.text for e in tree.findall('.//locationCode')]
print(locationCodes)

['STK.US', 'STK.US.MAJOR', 'STK.NYSE', 'STK.AMEX', 'STK.ARCA', 'STK.NASDAQ', 'STK.NASDAQ.NMS', 'STK.NASDAQ.SCM', 'STK.BATS', 'STK.US.MINOR', 'STK.PINK', 'STK.OTCBB', 'ETF.EQ.US', 'ETF.EQ.US.MAJOR', 'ETF.EQ.ARCA', 'ETF.EQ.NASDAQ.NMS', 'ETF.EQ.BATS', 'ETF.FI.US', 'ETF.FI.US.MAJOR', 'ETF.FI.ARCA', 'ETF.FI.NASDAQ.NMS', 'ETF.FI.BATS', 'FUT.US', 'FUT.GLOBEX', 'FUT.ECBOT', 'FUT.IPE', 'FUT.NYBOT', 'FUT.NYMEX', 'FUT.NYSELIFFE', 'FUT.CFE', 'FUT.CFECRYPTO', 'FUT.CMECRYPTO', 'FUT.ICECRYPTO', 'IND.US', 'BOND.WW', 'BOND.US', 'BOND.EU.EURONEXT', 'BOND.CD.US', 'BOND.AGNCY.US', 'BOND.GOVT.US', 'BOND.MUNI.US', 'BOND.GOVT.NON-US', 'BOND.GOVT.US.NON-US', 'BOND.GOVT.EU.EURONEXT', 'BOND.GOVT.HK.SEHK', 'SLB.PREBORROW', 'FUND.ALL', 'FUND.US', 'FUND.NON-US', 'STK.NA', 'STK.NA.CANADA', 'STK.NA.TSE', 'STK.NA.VENTURE', 'STK.NA.MEXI', 'FUT.NA', 'FUT.NA.CDE', 'FUT.NA.MEXDER', 'SSF.NA', 'SSF.NA.MEXDER', 'STK.EU', 'STK.EU.VSE', 'STK.EU.SBF', 'STK.EU.IBIS', 'STK.EU.IBIS-XETRA', 'STK.EU.IBIS-EUSTARS', 'STK.EU.IBIS-USST

In [11]:
ib.disconnect()